In [1]:
import wikipedia
import nltk
import re
from pymystem3 import Mystem
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# for pretty output
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [3]:
pd.set_option('max_colwidth', 80)
wikipedia.set_lang('ru')
tokenizer = nltk.RegexpTokenizer(r"\w+")
mystem = Mystem()

Факты

In [4]:
facts = [
    "Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.",
    "Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их тщательно дезинфицировали.",
    "В России акции протеста проходят не только на площадях, но и на поездах."
]

Название статей Википедии, на которые ссылаются факты

In [5]:
wikipedia_pages = {
    1: ["Шмидт, Генрих (врач)"],
    2: ["Ходоки у В. И. Ленина", "Дезинфекция"],
    3: ["Проезд снаружи поездов"]
}

### Скачивание и обработка статей из Википедии

In [6]:
def get_page_text(pagename):
    page = wikipedia.page(pagename)
    return page.content

In [7]:
def remove_trash(page_text):
    page_text = re.sub('\s', ' ', page_text)
    page_text = re.sub('[А-Я]\.', '', page_text)
    page_text = re.sub(r'\=\= Примечания[\w\s\=]*', '', page_text)
    page_text = page_text.replace('.', '. ')
    page_text = page_text.replace('\n', ' ')
    headers = re.findall(r"\=\=.*?\=\=", page_text)
    for header in headers:
        page_text = page_text.replace(header, '')
    
    return page_text

In [8]:
documents = []

for num in wikipedia_pages.keys():
    for page in wikipedia_pages[num]:
        page_text = remove_trash(get_page_text(page))
        sentences = nltk.sent_tokenize(page_text, language="russian")
        documents.extend(sentences)

In [9]:
# добавление фактов
documents.extend(facts)

In [10]:
wiki = pd.DataFrame(documents)

In [11]:
wiki.columns = ['document']

In [12]:
wiki.head()

,document
0,"Эрнст Генрих Шмидт (нем. Ernst Heinrich Schmidt; 27 марта 1912, Альтенбург,..."
1,Генрих Шмидт родился 27 марта 1912 года в Альтенбурге.
2,В 1933 году был зачислен в СС (№ 23 069).
3,В 1937 году получил степень доктора медицинских наук в Лейпцигском университ...
4,После начала Второй мировой войны работал в лазарете Ваффен С С 1941 года с...


### Лемматизация 

In [13]:
def lemmatize(sentence):
    return ''.join((mystem.lemmatize(' '.join(tokenizer.tokenize(sentence))))).replace('\n', '')

In [14]:
wiki['lemmatized_document'] = wiki.document.apply(lemmatize)

In [15]:
wiki.tail()

,document,lemmatized_document
561,Дата обращения 7 декабря 2012.,дата обращение 7 декабрь 2012
562,Архивировано 27 декабря 2012 года.,архивировать 27 декабрь 2012 год
563,"Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправда...",нацистский врач эсэсовец работать в шесть концлагерь быть дважды оправдывать...
564,"Историк утверждает, что, прежде чем допустить крестьян на личный прием к вож...",историк утверждать что прежде чем допускать крестьянин на личный прием к вож...
565,"В России акции протеста проходят не только на площадях, но и на поездах.",в россия акция протест проходить не только на площадь но и на поезд


In [16]:
def tf_idf_vectorizer(corpus):
    
    tf = []
    
    vocabulary = np.unique([word for word in ' '.join(corpus).split(' ') if len(word) != 0])
    
    corpus_length = len(corpus)
    vocabulary_length = len(vocabulary)
    
    def idf_computing(df):
        return np.log10(corpus_length / df)

    vectorized_idf = np.vectorize(idf_computing)
    
    inverse_vocabulary = {v: k for k, v in dict(enumerate(vocabulary)).items()}
    
    df = np.zeros(vocabulary_length, dtype=int)
    
    for document in corpus:
        document_vector = np.zeros(vocabulary_length, dtype=int)
        for word in document.split(' '):
            if word != '':
                document_vector[inverse_vocabulary[word]] += 1
                
        for word in np.unique(document.split(' ')):
            if word != '':
                df[inverse_vocabulary[word]] += 1
        
        tf.append(document_vector)
        
    idf = vectorized_idf(df)
    
    tf = [tf_elem * idf for tf_elem in tf]
    
    tf = np.stack( tfidf_weight, axis=0 )
    
    return tf

In [18]:
tfidf_weight = tf_idf_vectorizer(wiki['lemmatized_document'])

NameError: name 'tfidf_weight' is not defined

In [ ]:
nn_cosine = NearestNeighbors(metric='cosine')
nn_cosine.fit(tfidf_weight)

In [ ]:
facts_indexes = [563, 564, 565]

In [ ]:
def get_nearest_documents(index):
    cosine, indices = nn_cosine.kneighbors(tfidf_weight[index].reshape(1, -1), n_neighbors = 11)

    neighbors_cosine = pd.DataFrame({'cosine': cosine.flatten(), 'id': indices.flatten()})

    nearest_documents = (wiki.\
                    merge(neighbors_cosine, right_on = 'id', left_index = True).\
                    sort_values('cosine')[['id', 'document', 'cosine']])

    return nearest_documents

def print_results_for_document(index):
    nearest_documents = get_nearest_documents(index)
    documents = nearest_documents['document'].values
    cosines = nearest_documents['cosine'].values
    
    fact = documents[0]
    
    printmd("Факт: " + "**" + fact + "**")
    printmd("Близкие документы:")
    
    for doc, cosine in zip(documents[1:], cosines[1:]):
        printmd(f"* [{cosine:.2f}] {doc}")

In [ ]:
print_results_for_document(facts_indexes[0])

In [ ]:
print_results_for_document(facts_indexes[1])

In [ ]:
print_results_for_document(facts_indexes[2])